In [1]:
from imutils.video import FileVideoStream
import imutils
import dlib
import cv2
import numpy as np
from imutils import face_utils

In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [3]:
def shape_to_np(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = np.zeros((shape.num_parts, 2), dtype=dtype)

	# loop over all facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, shape.num_parts):
		coords[i] = (shape.part(i).x, shape.part(i).y)

	# return the list of (x, y)-coordinates
	return coords

In [4]:
from scipy.spatial import distance as dist

def eye_aspect_ratio(eye):
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	C = dist.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

def smile(mouth):
    A = dist.euclidean(mouth[3], mouth[9])
    B = dist.euclidean(mouth[2], mouth[10])
    C = dist.euclidean(mouth[4], mouth[8])
    avg = (A+B+C)/3
    D = dist.euclidean(mouth[0], mouth[6])
    mar=avg/D
    return mar

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

EYE_AR_THRESH = 0.3

### predict smile and blink

In [5]:
vs = FileVideoStream('ex2.mp4').start()

In [6]:
while vs.running():
    img = vs.read()
    img = imutils.resize(img, width=600)
    img_g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dets = detector(img_g, 0)
    for k, d in enumerate(dets):
        
        (x, y, w, h) = face_utils.rect_to_bb(d)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        #Get the landmarks/parts for the face in box d.
        shape = predictor(img, d)
        shape = shape_to_np(shape)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        if ear < EYE_AR_THRESH:
            cv2.putText(img, 'Blink', (x,abs(y-10)),0,0.01 * h,(255,255,255),2)
        
        mouth = shape[mStart:mEnd]
        mar= smile(mouth)
        if mar > 0.37:
            cv2.putText(img, 'Smile', (x,abs(y+h+10)),0,0.01 * h,(255,255,255),2)
        
        #Draw the face landmarks on the screen.
        for (x, y) in shape:
            cv2.circle(img, (x, y), 1, (0, 0, 255), -1)
    cv2.imshow('vid', img)
    if cv2.waitKey(5) == 13:
        break
cv2.destroyAllWindows()
vs.stop()

AttributeError: 'NoneType' object has no attribute 'shape'